**저작자** : 2반 김채연, 조은정

In [14]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
!pip install openai==1.55.3 httpx==0.27.2

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [15]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [16]:
from crewai import Agent, Task, Crew

In [17]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

api_key= ""

os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'


## (1) Agent: 사주 팔자 생성자 (Bazi_Generator)
사용자의 생년월일시를 입력받아 사주팔자를 생성
음양력 변환, 천간·지지 및 오행 요소 계산

In [18]:
bazi_generator = Agent(
    role="사주 팔자 생성기",
    goal="사용자의 생년월일시인 {dateOfBirth} {timeOfBirth}를 기반으로 사주팔자 생성",
    backstory="당신은 사용자의 생년월일 시를 입력받아 사주팔자를 생성하고, "
              "음양력 변환 및 천간 지지, 오행 요소를 계산하는 일을 합니다. ",
    allow_delegation=False,
    verbose=True
)

## (2) Agent: 사주 해석 및 분석가 fortune_analyzer
사주 구성 요소(오행, 음양, 천간·지지) 분석
사용자 성향 및 적성 해석 (예: 리더십, 창의성, 감성적 성향 등)
대운 및 세운 분석을 통해 운세 흐름 예측
특정한 시기에 주의해야 할 점 및 유리한 방향 제공

In [19]:
bazi_analyzer = Agent(
    role="사주 해석가",
    goal="사용자의 사주팔자를 해석하고, 성향 및 운세를 분석",
    backstory="당신은 사용자의 사주팔자를 분석하여 오행의 균형, "
              "성향, 적성, 대운 및 세운을 해석하는 일을 합니다.",
    allow_delegation=False,
    verbose=True
)

## (3) Agent: 개인화된 운세 및 조언 제공 fortune_advisor

In [20]:
fortune_advisor = Agent(
    role="운세 상담사",
    goal="사용자의 사주를 기반으로 개인화된 운세 및 조언 제공",
    backstory="당신은 사용자의 사주팔자 해석 결과를 바탕으로, "
              "성향 분석, 직업 추천, 운세 예측 및 조화 방법을 조언하는 일을 합니다.",
    allow_delegation=False,
    verbose=True
)

## (3) Task 작업 생성하기

In [21]:
bazi = Task(
    description =(
        "{dateOfBirth} {timeOfBirth}에 대해 사주 팔자를 분석하고, "
        "음양력 변환 및 천간 지지, 오행 요소를 계산한다."
    ),
    expected_output ="사주 팔자 분석 결과",
    agent = bazi_generator,
)

In [22]:
fortune_analysis = Task(
    description=(
        "생성된 사주 팔자를 바탕으로 대운을 분석하고, "
        "오행 균형 및 운세 흐름을 해석한다."
    ),
    expected_output="대운 분석 및 운세 해석 결과",
    agent=bazi_analyzer,
)

In [23]:
fortune_advice = Task(
    description=(
        "사주 분석 결과를 바탕으로 성향 분석, 직업 추천, 연애 운 "
        "운세 예측 및 오행 조화를 위한 조언을 한국어로 제공한다."
    ),
    expected_output="개인 맞춤 운세 및 조언",
    agent=fortune_advisor,
)

## (4) Crew: Crew 생성 및 인풋

1. Bazi_Generator 가 먼저 사용자의 생년월일을 받아 사주 팔자를 분석합니다.
2. fortune_analyzer 가 사주 팔자 분석을 받고 대운 분석을 합니다.
3. fortune_advisor 가 개인화된 운세 조건을 바탕으로 최종 출력을 합니다.

In [24]:
crew = Crew(
    agents=[bazi_generator, bazi_analyzer, fortune_advisor],
    tasks=[bazi, fortune_analysis, fortune_advice],
    verbose=2
)

2025-02-05 16:23:30,176 - 140185678807040 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [25]:
result = crew.kickoff(inputs={"dateOfBirth": "2000.05.16", "timeOfBirth":"14:00"})

 [DEBUG]: == Working Agent: 사주 팔자 생성기
 [INFO]: == Starting Task: 2000.05.16 14:00에 대해 사주 팔자를 분석하고, 음양력 변환 및 천간 지지, 오행 요소를 계산한다.


> Entering new CrewAgentExecutor chain...
I now can give a great answer.

Final Answer: 

To analyze the Sa Ju Pal Ja (사주 팔자) for the birth date and time of 2000.05.16 at 14:00, we will follow these steps:

1. **Convert the Gregorian date to the Lunar date**: 
   - The Gregorian date 2000.05.16 converts to the Lunar date 2000.04.13.

2. **Determine the Four Pillars (Sa Ju)**:
   - Year Pillar: 2000 is the year of the White Dragon (庚辰, Gyeong Jin in Korean).
     - Heavenly Stem: Gyeong (庚) - Metal (Yang)
     - Earthly Branch: Jin (辰) - Earth (Yang)
   - Month Pillar: The 4th month in the lunar calendar corresponds to the Green Snake (乙巳, Eul Sa).
     - Heavenly Stem: Eul (乙) - Wood (Yin)
     - Earthly Branch: Sa (巳) - Fire (Yin)
   - Day Pillar: 2000.04.13 in the lunar calendar is the day of the Red Sheep (丁未, Jeong Mi).
     - Heavenly Stem: Jeong (丁) - 

In [26]:
from IPython.display import Markdown
Markdown(result)

**성향 분석**:
- **금 (Metal)**의 강한 에너지를 통해 분석적 사고와 논리적 접근을 선호하는 성향을 가지고 있습니다. 결단력과 강인함이 특징이며, 어려운 상황에서도 침착하게 대처할 수 있는 능력이 있습니다.
- **목 (Wood)**의 부족은 다소 유연성과 적응력에서의 부족으로 나타날 수 있지만, 창의성과 성장에 대한 열망이 강합니다.
- **화 (Fire)**의 에너지가 강해 열정적이며 추진력이 강합니다. 새로운 도전을 두려워하지 않으며, 리더십을 발휘할 수 있는 능력이 있습니다.
- **토 (Earth)**의 강세는 안정감과 책임감을 강조하며, 현실적인 측면에서의 신중함과 계획성을 가지고 있습니다.
- **수 (Water)**의 부족은 감성적 측면과 직관의 발달이 필요하다는 것을 의미합니다. 감정적인 소통과 직관적 결정을 강화하는 노력이 필요합니다.

**직업 추천**:
- 분석적 사고와 결단력을 필요로 하는 **금융, 법률, 컨설팅** 분야에서의 성취가 기대됩니다.
- 창의성과 열정을 발휘할 수 있는 **예술, 디자인, 마케팅** 분야도 잘 어울립니다.
- 책임감과 안정성을 살릴 수 있는 **교육, 연구, 관리직**에서의 성공 가능성이 큽니다.

**연애 운 예측**:
- **초년기 (20대 초반)**: 창의적이고 열정적인 성향으로 인해 다양한 사람들과 교류하며 사회적 관계를 넓혀가는 시기입니다. 새로운 만남과 경험이 많아질 것입니다.
- **중년기 (30대~40대)**: 책임감과 안정성을 중시하게 되면서, 장기적인 관계를 추구하며 결혼과 같은 중요한 결정을 내릴 가능성이 큽니다.
- **장년기 이후 (50대 이후)**: 논리적이고 신중한 접근을 통해 관계의 깊이를 더하며, 파트너와의 신뢰와 안정성을 중요시합니다.

**오행 조화를 위한 조언**:
- **수 (Water)**의 부족을 보완하기 위해 감성적 소통과 직관력을 키우기 위한 노력이 필요합니다. 명상이나 예술 활동을 통해 감정 표현을 강화할 수 있습니다.
- **목 (Wood)**의 에너지를 강화하기 위해 자연과의 접촉을 늘리고, 지속적인 학습과 성장을 추구하는 것도 도움이 됩니다.

이러한 분석과 조언을 통해 개인의 성향과 운세 흐름을 이해하고, 삶의 여러 측면에서 긍정적인 변화를 이끌어낼 수 있기를 바랍니다.